In [1]:
import torch
import torchvision
device = torch.device('cuda')

In [2]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('road_following_model_trt.pth')) # well trained road following model

model_trt_collision = TRTModule()
model_trt_collision.load_state_dict(torch.load('best_model1_trt.pth')) # well trained collision avoidance model

<All keys matched successfully>

In [3]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [4]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg

#camera = Camera.instance(width=224, height=224)
camera = CSICamera(width=224, height=224, capture_fps=10)
# camera = USBCamera(width=224, height=224)

camera.running = True
image = widgets.Image(format='jpeg', width=224, height=224)
blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')
speed_slider = widgets.FloatSlider(description='speed', min=-1.0, max=1.0, value=0.0, step=0.01, orientation='horizontal')
steer_slider = widgets.FloatSlider(description='steer', min= -1.0, max=1.0, value=0.0, step=0.01, orientation='horizontal')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)



display(widgets.VBox([widgets.HBox([image, blocked_slider]), speed_slider, steer_slider]))

In [11]:
from jetracer.nvidia_racecar import NvidiaRacecar

car = NvidiaRacecar()


In [9]:
import torch.nn.functional as F
import time
import numpy as np

STEERING_GAIN = 0.8
STEERING_BIAS = -0.15

def update(change):
    global blocked_slider
    x = change['new'] 
    x = preprocess(x)
    y = model_trt_collision(x)
    #print(y)
    
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    #print(y)
    
    prob_blocked = float(y.flatten()[0])
    #print(prob_blocked)
    
    blocked_slider.value = prob_blocked
    
    if prob_blocked < 0.88:
        car.throttle = 0.170
        #image = preprocess(change['new']).half()
        #image = preprocess(image).half()
        image = change['new']
        xy = model_trt(preprocess(image)).detach().float().cpu().numpy().flatten()
        #output = model_trt(image).detach().cpu().numpy().flatten()
        #x = float(output[0])
        z = xy[0]
        car.steering = z * STEERING_GAIN + STEERING_BIAS
    else:
        car.throttle = 0.0
    
   # time.sleep(0.001)
        
update({'new': camera.value})
'''
while True:
    global blocked_slider
    image = camera.read()
    x = preprocess(image)
    y = model_trt_collision(x)
    y = F.softmax(y, dim=1)
    prob_blocked = float(y.flatten()[0])
    blocked_slider.value = prob_blocked
    
    if prob_blocked < 0.78:
        car.throttle = 0.158
        img = preprocess(image).half()
        output = model_trt(img).detach().cpu().numpy().flatten()
        z = float(output[0])
        car.steering = z * STEERING_GAIN + STEERING_BIAS
    else:
        car.throttle = 0.0'''


'\nwhile True:\n    global blocked_slider\n    image = camera.read()\n    x = preprocess(image)\n    y = model_trt_collision(x)\n    y = F.softmax(y, dim=1)\n    prob_blocked = float(y.flatten()[0])\n    blocked_slider.value = prob_blocked\n    \n    if prob_blocked < 0.78:\n        car.throttle = 0.158\n        img = preprocess(image).half()\n        output = model_trt(img).detach().cpu().numpy().flatten()\n        z = float(output[0])\n        car.steering = z * STEERING_GAIN + STEERING_BIAS\n    else:\n        car.throttle = 0.0'

In [10]:
camera.observe(update, names='value')  # this attaches the 'update' function to the 'value' traitlet of our camera

In [8]:
import time

camera.unobserve(update, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

# 

In [85]:
camera_link.unlink()

TypeError: 'NoneType' object is not subscriptable